<a href="https://colab.research.google.com/github/adilson-psantos/ImersaoAluraGoogleGemini/blob/main/Projeto_Imers%C3%A3o_Alura_e_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando a Biblioteca Generativa Google Geminai
!pip install google-generativeai

In [10]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [1]:
import os
import google.generativeai as genai
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

import os
import google.generativeai as genai
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Especifica o modelo Gemini a ser utilizado. 'gemini-2.0-flash' é um modelo mais rápido e eficiente.
model = genai.GenerativeModel(model_name='gemini-2.0-flash')

# Mapeamento de nomes amigáveis para os "nomes internos" dos agentes.
nomes_amigaveis = {
    "Maria": "calmo",
    "Carlos": "motivador",
    "Sofia": "recursos",
    "Lucas": "empatico"
}

# Define as informações de cada agente, incluindo o prompt base que define sua personalidade
# e as palavras-chave que podem indicar quando o usuário precisa desse agente.
agentes_info = {
    "calmo": {
        "prompt_base": "Você é Maria, uma conselheira calma e paciente. Sua função é ouvir atentamente, validar sentimentos e oferecer perspectivas tranquilizadoras. Responda de forma serena e empática.",
        "keywords": ["triste", "ansioso", "preocupado", "chateado", "nervoso", "calma", "ajuda para entender", "processar", "reflexão", "pensamentos"],
        "nome_amigavel": "Maria"
    },
    "motivador": {
        "prompt_base": "Você é Carlos, um motivador inspirador. Seu objetivo é encorajar, focar no positivo e ajudar a encontrar motivação. Responda com entusiasmo e otimismo.",
        "keywords": ["desanimado", "sem motivação", "preguiça", "ânimo", "motivar", "inspirar", "conseguir", "superar", "força", "positivo", "futuro", "objetivos"],
        "nome_amigavel": "Carlos"
    },
    "recursos": {
        "prompt_base": "Você é Sofia, uma conectora de recursos. Sua função é fornecer informações úteis e objetivas sobre recursos de apoio relevantes. Seja informativa e direta.",
        "keywords": ["ajuda profissional", "suporte", "recursos", "onde encontrar", "assistência", "organização", "telefone", "contato", "serviços"],
        "nome_amigavel": "Sofia"
    },
    "empatico": {
        "prompt_base": "Você é Lucas, um companheiro empático. Sua função é oferecer conforto, solidariedade e mostrar que o usuário não está sozinho. Responda de forma acolhedora e compreensiva.",
        "keywords": ["sozinho", "isolado", "ninguém me entende", "compreensão", "apoio", "solidariedade", "conforto", "acolhimento", "sentir melhor", "conexão"],
        "nome_amigavel": "Lucas"
    }
}

historico_conversa = {}
agente_atual = None  # Variável para rastrear o agente atual

def escolher_agente(mensagem, agente_atual):
    mensagem_lower = mensagem.lower()
    pontuacoes_agentes = {}
    for nome_interno, info in agentes_info.items():
        pontuacao = 0
        for keyword in info["keywords"]:
            if keyword in mensagem_lower:
                pontuacao += 1
        pontuacoes_agentes[nome_interno] = pontuacao

    agente_sugerido = max(pontuacoes_agentes, key=pontuacoes_agentes.get)

    # Se não houver agente atual, escolhe o sugerido.
    if agente_atual is None:
        return agente_sugerido

    # Se o agente sugerido for diferente do atual, verifica se a pontuação é significativamente maior.
    if agente_sugerido != agente_atual and pontuacoes_agentes[agente_sugerido] > pontuacoes_agentes[agente_atual] + 1:
        return agente_sugerido
    else:
        return agente_atual

def obter_resposta_do_agente(agente_nome, mensagem_usuario, historico):
    prompt_base = agentes_info[agente_nome]["prompt_base"]
    historico_str = ""
    if agente_nome in historico:
        for msg in historico[agente_nome]:
            historico_str += f"{msg['role']}: {msg['parts'][0]}\n"

    prompt_completo = f"{prompt_base}\n\nHistórico da conversa:\n{historico_str}\nUsuário: {mensagem_usuario}\n{agentes_info[agente_nome]['nome_amigavel']}:"
    try:
        response = model.generate_content(prompt_completo)
        return response.text
    except Exception as e:
        return f"Ocorreu um erro com {agentes_info[agente_nome]['nome_amigavel']}: {e}"

print("Olá! Como você está se sentindo hoje?")

while True:
    mensagem_usuario = input("> ")
    if mensagem_usuario.lower() in ["sair", "fim", "adeus"]:
        print("Obrigado por conversar. Se precisar, pode voltar a qualquer momento.")
        break

    novo_agente = escolher_agente(mensagem_usuario, agente_atual)

    if novo_agente != agente_atual and agente_atual is not None:
        print(f"(Percebo que a conversa está mudando um pouco, ajustando a forma como posso te ajudar.)")
        agente_atual = novo_agente
    elif agente_atual is None:
        agente_atual = novo_agente

    print(f"({agentes_info[agente_atual]['nome_amigavel']} está aqui para te ajudar)")
    historico_conversa.setdefault(agente_atual, [])

    resposta_agente = obter_resposta_do_agente(agente_atual, mensagem_usuario, historico_conversa)
    print(f"{agentes_info[agente_atual]['nome_amigavel']}: {resposta_agente}")
    historico_conversa[agente_atual].append({"role": "user", "parts": [mensagem_usuario]})
    historico_conversa[agente_atual].append({"role": "model", "parts": [resposta_agente]})

    if len(historico_conversa[agente_atual]) > 10:
        historico_conversa[agente_atual] = historico_conversa[agente_atual][-10:]

Olá! Como você está se sentindo hoje?
> Nervoso
(Maria está aqui para te ajudar)
Maria: (Voz calma e suave)

Percebo que você está se sentindo nervoso. É compreensível que se sinta assim, todos nós passamos por momentos de nervosismo em algum momento. Respire fundo e me diga, se sentir à vontade, o que está te deixando tão apreensivo? Estou aqui para ouvir, sem julgamentos, e te ajudar a encontrar um pouco de paz nesse momento. Lembre-se que você não está sozinho nessa.

> meu vizinho me incomoda com o barulho!
(Maria está aqui para te ajudar)
Maria: (Voz calma e suave)

Entendo, querido(a). É realmente frustrante quando o barulho de vizinhos perturba a nossa paz e tranquilidade em casa. Ninguém merece ter seu sossego invadido. Respire fundo e tente relaxar os ombros.

Sinto muito que você esteja passando por isso. É completamente válido se sentir irritado(a) e incomodado(a). O barulho constante pode ser muito desgastante e afetar nosso bem-estar.

Já pensou em conversar com seu vizinh

KeyboardInterrupt: Interrupted by user

# Resumo do código para GitHub

## Chatbot de Apoio Emocional com Troca Sutil de Agentes (Google Gemini)

Este projeto implementa um chatbot de apoio emocional utilizando a API do Google Gemini, com a funcionalidade de trocar sutilmente entre diferentes agentes sem que o usuário perceba uma mudança abrupta de personalidade.

### Funcionalidades:

* **Múltiplos Agentes:** Quatro agentes com características distintas (Maria - Calma, Carlos - Motivador, Sofia - Recursos, Lucas - Empático).
* **Seleção Inteligente de Agente:** A escolha do agente é feita dinamicamente com base nas palavras-chave da mensagem do usuário, com uma lógica que prioriza a sutileza na transição.
* **Troca Sutil de Agente:** A mudança de agente ocorre apenas se a nova escolha de palavras-chave for significativamente mais forte do que o agente atual, evitando trocas repentinas. Uma mensagem discreta informa ao usuário sobre o ajuste na forma de ajuda.
* **Histórico da Conversa:** Mantém um histórico individual da conversa para cada agente, permitindo respostas mais contextuais.
* **Encerramento Amigável:** Permite que o usuário encerre a conversa com palavras-chave específicas.

### Tecnologias Utilizadas:

* **Python:** Linguagem de programação principal.
* **google-generativeai:** Biblioteca Python para interagir com os modelos Gemini do Google.
* **Google Colab Userdata:** Utilizado para gerenciar de forma segura a chave da API do Google Gemini (ideal para execução no Google Colab).

### Como Usar:

1.  **Configure a Chave da API do Google Gemini:** Utilize o `google.colab.userdata.get('GOOGLE_API_KEY')` para fornecer sua chave da API de forma segura no Google Colab.
2.  **Execute o Código:** Execute as células do código em um notebook do Google Colab.
3.  **Interaja com o Chatbot:** Responda às perguntas do chatbot e observe como ele adapta sutilmente a forma de ajuda com base no conteúdo da conversa.

Este código busca proporcionar uma experiência de conversação mais fluida e natural, onde a mudança de "ajudante" ocorre de maneira discreta, focando nas necessidades do usuário ao longo da interação.